In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
from tqdm import tqdm
from pylab import *

In [ ]:

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.KMNIST(root='../../data',      
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.KMNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)



In [ ]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,num_classes)
        
    def forward(self,x):
        out=self.fc1(x)
        out=self.relu(out)
        out=self.fc2(out)
        return out
    
model=NeuralNet(input_size,hidden_size,num_classes).to(device)

#loss and optimiser
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)        

In [ ]:

class Kuzushiji(nn.Module):
    def __init__(self):
        super(Kuzushiji, self).__init__()
        self.pad1 = nn.ZeroPad2d((2, 2))
        
        self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(7, 7), stride=(1, 1), bias=False), 
        nn.BatchNorm2d(num_features=32),
        nn.MaxPool2d(kernel_size=(2,2)),
        nn.ReLU()
        )
    
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5, 5), stride=(1, 1), bias=False), 
        nn.BatchNorm2d(num_features=64),
        nn.ReLU()
        )
        
        self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), bias=False), 
        nn.BatchNorm2d(num_features=128),
        nn.MaxPool2d(kernel_size=(2,2)),
        nn.ReLU()
        )
        
        self.fc1 = nn.Sequential(
        nn.Linear(in_features=768, out_features=512),
        nn.ReLU()
        )
        self.dropout1 = nn.Dropout(p=0.2)
        
        self.fc2 = nn.Sequential(
        nn.Linear(in_features=512, out_features=10),
        nn.Softmax()
        )
    def forward(self, x):
        x = self.pad1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(x.shape[0], -1)
       # print(x.shape)
        x = self.fc1(x)
        x = self.dropout1(x)
        out = self.fc2(x)
        
        return out

In [ ]:
my_model=Kuzushiji().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(my_model.parameters(),lr=learning_rate)

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 1, 28,28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = my_model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
torch.max(images[1])

In [ ]:
# Normal Model Testing

with torch.no_grad():
    correct=0
    total=0
    for images,labels in test_loader:
        images=images.reshape(-1,1,28,28).to(device)
        labels=labels.to(device)
        outputs=my_model(images)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
        
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
for param in my_model.parameters():
    param.requires_grad=False

In [ ]:
# Targeted FGSM attack 

epsilon=[0.0001,0.0003,0.001,0.003,0.1,0.3,0.5]

for value in epsilon:
    for k in range(10):
        l=[]
        sign=[]
        #actual=[]
        for i,(images,labels) in enumerate(test_loader):
            copyOf_images=images.reshape(-1,1,28,28).to(device)
            copyOf_images.requires_grad=True
            target_class=(labels+k)%10  #missing classifying it with 10 classes though it has to be correct at least once
            target_class=target_class.to(device)

            #forward pass
            outputs=my_model(copyOf_images)
            loss=criterion(outputs,target_class)

            #backward and optimize
            optimizer.zero_grad()
            loss.backward()
            c=copyOf_images.grad.sign()
            #actual.append((images,labels))
            copyOf_images=copyOf_images-(value*c) 
            copyOf_images=torch.clamp(copyOf_images,0,1)
            sign.append(c)
            l.append((copyOf_images,labels))

        with torch.no_grad():
            correct=0
            total=0
            for images,labels in l:
                images=images.reshape(-1,1,28,28).to(device)
                labels=labels.to(device)
                outputs=my_model(images)
                _,predicted=torch.max(outputs.data,1)
                total+=labels.size(0)
                correct+=(predicted==labels).sum().item()
            print("accuracy of the network on the 10000 test images for epsilon value {}:{} %".format(value,100*correct/total)) 
    

In [ ]:
# Untargeted FGSM attack 

l=[]
sign=[]
actual=[]
epsilon=[0.0001,0.0003,0.001,0.003,0.1,0.2,0.3,0.4,0.5]

for value in epsilon:
   # for k in range(10):
        l=[]
        sign=[]
        #actual=[]
        for i,(images,labels) in enumerate(test_loader):
            copyOf_images=images.reshape(-1,1,28,28).to(device)
            copyOf_images.requires_grad=True
            #target_class=(labels+k)%10  #missing classifying it with 10 classes though it has to be correct at least once
            target_class=labels
            target_class=target_class.to(device)
            
            #forward pass
            outputs=my_model(copyOf_images)
            loss=criterion(outputs,target_class)

            #backward and optimize
            optimizer.zero_grad()
            loss.backward()
            c=copyOf_images.grad.sign()
            #actual.append((images,labels))
            copyOf_images=copyOf_images+(value*c) 
            copyOf_images=torch.clamp(copyOf_images,0,1)
            sign.append(c)
            l.append((copyOf_images,labels))

        with torch.no_grad():
            correct=0
            total=0
            for images,labels in l:
                images=images.reshape(-1,1,28,28).to(device)
                labels=labels.to(device)
                outputs=my_model(images)
                _,predicted=torch.max(outputs.data,1)
                total+=labels.size(0)
                correct+=(predicted==labels).sum().item()
            print("accuracy of the network on the 10000 test images for epsilon value {}:{} %".format(value,100*correct/total)) 
    

In [ ]:
#checking the accuracy after FGSM attack

with torch.no_grad():
    correct=0
    total=0
    for images,labels in l:
        images=images.reshape(-1,1,28,28).to(device)
        labels=labels.to(device)
        outputs=my_model(images)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
    print("accuracy of the network on the 10000 test images:{} %".format(100*correct/total))

# Carlini Wagner Attack



**> Following are just snippet tests,they don't mean anything as such
**

In [ ]:
im,_=l[0]
im=im.data.cpu().numpy()

In [ ]:
old_im,_=actual[0]
old_im=old_im.data.cpu().numpy()

In [ ]:
im=im.reshape(100,28,28)
old_im=old_im.reshape(100,28,28)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(old_im[2])

In [ ]:
plt.imshow(im[2])

In [ ]:
#these ae just snippet tests they don't mean anything now
subplot(1,2,1)
title('adversarial example')
imshow(im[2],cmap='gray')
subplot(1,2,2)
title('actual sample')
imshow(old_im[2],cmap='gray')

In [ ]:
image1=torch.from_numpy(im[1])
img_t1=image1.reshape(-1,1,28,28).to(device)
outputs=my_model(img_t1)
_,predicted=torch.max(outputs.data,1)
print(predicted)

image2=torch.from_numpy(old_im[1])
img_t2=image2.reshape(-1,1,28,28).to(device)
outputs=my_model(img_t2)
_,predicted=torch.max(outputs.data,1)
print(predicted)

**Training a GAN **




In [ ]:
g_input_size=10
g_hidden_size=5
g_output_size=

d_input_size=
d_hidden_size=10
d_output_size=1
minibatch_size=d_input_size 

generator_activation_function=torch.tanh
discriminator_activation_function=torch.sigmoid

In [ ]:
def get_generator_input_sampler():
    return lambda n: torch.Tensor(np.random.normal(0,1,(g_input_size,1)))  #whatever is the size of the random input noise here 50*1

In [ ]:
class Generator(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,f):
        super(Generator,self).__init__()
        self.map1=nn.Linear(input_size,hidden_size)
        self.map2=nn.Linear(hidden_size,hidden_size)
        self.map3=nn.Linear(hidden_size,output_size)
        self.f=f
    def forward(self,x):
        x=self.map1(x)
        x=self.f(x)
        x=self.map2(x)
        x=self.f(x)
        x=self.map3(x)
        return x

In [ ]:
def Discriminator(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,f):
        super(Discriminator,self).__init__()
        self.map1=nn.Linear(input_size,hidden_size)
        self.map2=nn.Linear(hidden_size,hidden_size)
        self.map3=nn.Linear(hidden_size,output_size)
        self.f=f
    def forward(self,x):
        x=self.f(self.map1(x))
        x=self.f(self.map2(x))
        x=self.f(self.map3(x))
        return x

In [ ]:
G=Generator(input_size=g_input_size,
           hidden_size=g_hidden_size,
           output_size=g_output_size,
           f=generator_activation_function)

D=Discriminator(input_size=d_input_size,
           hidden_size=d_hidden_size,
           output_size=d_output_size,
           f=discriminator_activation_function)


In [ ]:
Number_epochs=50
D_steps=5

for epoch in range(Number_epochs):
    for d_index in range(D_steps):
        #train D on real+fake
        D
        